In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df_Taian = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//Taian.csv', low_memory=False)
df_Z_N = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//Zhongli_N.csv', low_memory=False)
df_Z_S = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//Zhongli_S.csv', low_memory=False)
df_Xihu = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//Xihu.csv', low_memory=False)
df_Xilou = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//Xilou.csv', low_memory=False)
df_QingShui = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//QingShui.csv', low_memory=False)
df_Nantou = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//Nantou.csv', low_memory=False)
df_Guanxi = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//Guanxi.csv', low_memory=False)
df_Huko = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//Huko.csv', low_memory=False)
df_Zhongli = pd.concat([df_Z_N, df_Z_S])

In [3]:
df = pd.concat([df_Taian, df_Xihu, df_Zhongli, df_Xilou, df_QingShui, df_Nantou, df_Guanxi, df_Huko])

In [5]:
df.to_csv('C://Users//Administrator//Documents//code//data//SA_advanced//all.csv', encoding='utf-8')

In [4]:
df['ENTRY_TIME'] = pd.to_datetime(df['ENTRY_TIME'],infer_datetime_format=True)
df['EXIT_TIME'] = pd.to_datetime(df['EXIT_TIME'],infer_datetime_format=True)
df['O_TIME'] = pd.to_datetime(df['O_TIME'],infer_datetime_format=True)
df['D_TIME'] = pd.to_datetime(df['D_TIME'],infer_datetime_format=True)

In [6]:
gnty_df = pd.read_csv('C://Users//Administrator//Documents//code//data//SA_advanced//GANTRY_NAME.csv', low_memory=False)

In [7]:
gnty_cnty_df = pd.read_excel('C://Users//Administrator//Documents//code//data//SA_advanced//GATRY_CITY.xlsx')

In [8]:
def make_clean_gantry(x):
    x = x.strip()
    return x

In [9]:
def split_gantry_in(x) :
    var = x.split('-')[0]
    return var

In [10]:
def split_gantry_out(x) :
    var = x.split('-')[1]
    return var

In [18]:
gnty_cnty_df['GANTRY_ID'] = gnty_cnty_df['GANTRY_ID'].apply(make_clean_gantry)

In [11]:
gnty_df['GANTRY_ID'] = gnty_df['GANTRY_ID'].apply(make_clean_gantry)
gnty_df["in"] = gnty_df['GANTRY_NAME'].apply(split_gantry_in)
gnty_df["out"] = gnty_df['GANTRY_NAME'].apply(split_gantry_out)

In [12]:
# for each SA df, apply to ["O_CITY"]
def define_gantry_location_in(x):
    #location = ''
    for index, gantry in enumerate(gnty_df['GANTRY_ID']):
        if x == gantry :
            location = gnty_df.iloc[index, :]['in']
    return location        

In [13]:
# for each SA df, apply to ["D_CITY"]
def define_gantry_location_out(x):
    #location = ''
    for index, gantry in enumerate(gnty_df['GANTRY_ID']):
        if x == gantry :
            location = gnty_df.iloc[index, :]['out']
    return location     

In [19]:
# for each SA df, apply to ["O_CITY", "D_CITY"]
def define_gantry_location_country(x):
    #location = ''
    for index, gantry in enumerate(gnty_cnty_df['GANTRY_ID']):
        if x == gantry :
            location = gnty_cnty_df.iloc[index, :]['CODE_NAME']
    return location        

In [14]:
df['in_location'] = df['O_CITY'].apply(define_gantry_location_in)
df['out_location'] = df['D_CITY'].apply(define_gantry_location_out)

In [20]:
#df['in_location'] = df['O_CITY'].apply(define_gantry_location_in)
#df['out_location'] = df['D_CITY'].apply(define_gantry_location_out)
df['O_COUNTRY'] = df['O_CITY'].apply(define_gantry_location_country)
df['D_COUNTRY'] = df['D_CITY'].apply(define_gantry_location_country)

In [21]:
df.to_csv('C://Users//Administrator//Documents//code//data//SA_advanced//all_wOD.csv')

In [25]:
df.columns

Index(['SERVICE_AREA', 'BMS_TX_BATCH', 'DIRECTION_ID', 'ENTRY_TIME',
       'EXIT_TIME', 'STOP', 'STOP_DURATION', 'VEHICLE_ID', 'IDENTITY_ID',
       'Gender', 'MVDIS_CATEGORY', 'MANU_YEAR', 'MANU_MONTH', 'VEHICLE_AGE',
       'MVDIS_BRAND', 'BRAND', 'Holiday_cnt', 'WEEKDAY_CNT', 'TX_cnt',
       'T_MILEAGE', 'O_CITY', 'O_TIME', 'O_MILEAGE', 'D_CITY', 'D_TIME',
       'TRIP_DURATION', 'TRIP_DISTANCE', 'ENTRY_SITE', 'EXIT_SITE', 'NAME',
       'in_location', 'out_location', 'O_COUNTRY', 'D_COUNTRY'],
      dtype='object')

In [ ]:
(df.VEHICLE_ID.isin(df[df.STOP == 'P']['VEHICLE_ID'].unique()))

#### all of the cars, car brand

In [26]:
def sorting_car_brand(df) :
    brand_list = []
    categories = [31, 32, 41, 42, 5]
    car_dict = {}
    car_dfs = []
    for car in df['VEHICLE_ID'].unique() :
        
        #brand = pd.DataFrame(df[df.VEHICLE_ID == car].head(1))
        brand = df[df.VEHICLE_ID == car].head(1)
        
        brand_list.append(brand)
    brand_df = pd.concat(brand_list, axis=0)
    
    for category in categories :
        series = brand_df[(brand_df.MVDIS_CATEGORY == category)].groupby("BRAND").size().sort_values(0, ascending=False)[:20]
        tmp_df = pd.DataFrame(series).reset_index()
        car_dict['group_{0}'.format(category)] = tmp_df
        car_dfs.append(car_dict['group_{0}'.format(category)])
    result_df = pd.concat(car_dfs, axis=1)
    
    iterables = [['31', '32', '41', '42', '5'], ['BRAND', 'car_num']]
    col = pd.MultiIndex.from_product(iterables)
    result_df.columns = col
    return result_df

#### into SA, car brand

In [27]:
def sorting_car_brand_inSA(df) :
    
    categories = [31, 32, 41, 42, 5]
    car_dict = {}
    car_dfs = []
    brand_list = []
    for car in df['VEHICLE_ID'].unique() :
        
        #brand = pd.DataFrame(df[df.VEHICLE_ID == car].head(1))
        brand = df[(df.STOP == "P")&(df.VEHICLE_ID == car)].head(1)
        
        brand_list.append(brand)
    brand_df = pd.concat(brand_list, axis=0)
    
    for category in categories :
        series = brand_df[(brand_df.MVDIS_CATEGORY == category)].groupby("BRAND").size().sort_values(0, ascending=False)[:20]
        tmp_df = pd.DataFrame(series).reset_index()
        car_dict['group_{0}'.format(category)] = tmp_df
        car_dfs.append(car_dict['group_{0}'.format(category)])
    result_df = pd.concat(car_dfs, axis=1)
    
    iterables = [['31', '32', '41', '42', '5'], ['BRAND', 'car_num']]
    col = pd.MultiIndex.from_product(iterables)
    result_df.columns = col
    return result_df

In [ ]:
carbrand_in_df = sorting_car_brand_inSA(df)

In [28]:
def sorting_trip(df) :
    SA_name = ["Taian", "Zhongli", "Xihu", "Xilou", "Nantou", "QingShui", "Huko", "Guanxi"]
    SA_Cname = ['泰安', '中壢', '西湖', '西螺', '南投', '清水', '湖口', '關西']
    categories = [31, 32, 41, 42, 5]
    for idx, sa in enumerate(SA_Cname) :  # the first SA
        #writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
        categories_df = {}
        
        for category in categories :
            #writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
            var_dict = {}
            cnt_row = []
            for var in SA_Cname :
                var_dict['v_{0}'.format(var)] = 0
            
            unique_bms = df[(df.SERVICE_AREA == sa)&(df.STOP == 'P')&(df.MVDIS_CATEGORY == category)]['BMS_TX_BATCH'].unique()
            
            first_row = []
            second_row = []
            third_row = []
            col_name = []
            
            for bms in unique_bms :
                series = df[(df.BMS_TX_BATCH == bms)&(df.STOP == 'P')].groupby("SERVICE_AREA").size()
                cnt_df = pd.DataFrame(series).reset_index()
                length = cnt_df.shape[0]
                for p_name in [x for index, x in enumerate(SA_Cname) if index != idx] : # this list won't have the first SA itself 
                    for idx_cnt, i in cnt_df.iterrows():
                        if i.SERVICE_AREA == p_name :
                            var_dict["v_{0}".format(p_name)] = var_dict["v_{0}".format(p_name)] + i[0]
            # trip grouping start here
            #bms_df = df[(df.BMS_TX_BATCH.isin(unique_bms))]
            for sa2 in [x for index, x in enumerate(SA_Cname) if index != idx] : # sa2 should be the same with p_name
                col_name.append(sa2)
                select_vID = df[(df.BMS_TX_BATCH.isin(unique_bms))&(df.SERVICE_AREA == sa2)&(df.STOP == 'P')]['VEHICLE_ID'].unique()
                trip_series = df[df.VEHICLE_ID.isin(select_vID)].groupby(['in_location', 'out_location']).size().sort_values(0, ascending=False)[:3]
                trip_df = pd.DataFrame(trip_series).reset_index()
                
                for idx_trip, place in trip_df.iterrows():
                    place_string = "{0}-{1}".format(place['in_location'], place['out_location'])
                    trip_dict['{0}_place'.format(idx_trip)] = place_string
                    if idx_trip == 0 :
                        first_row.append(trip_dict['{0}_place'.format(idx_trip)])
                    elif idx_trip == 1 :
                        second_row.append(trip_dict['{0}_place'.format(idx_trip)])
                    elif idx_trip == 2 :
                        third_row.append(trip_dict['{0}_place'.format(idx_trip)])
                # collect the variables of car counting
                cnt_row.append(var_dict['v_{0}'.format(sa2)])
            result_df =  pd.DataFrame([cnt_row, first_row, second_row, third_row], columns = col_name)    
            categories_df['{0}_{1}'.format(sa, category)] = result_df
            
            writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
            categories_df['{0}_{1}'.format(sa, category)].to_excel(writer,'{0}_{1}_trip'.format(sa, category)) 
            writer.save()
        # saving excel
        #writer = pd.ExcelWriter('C://Users//Administrator//Documents//code//data//SA_advanced//analysis_result//{0}_{1}_all_trip.xlsx'.format(sa, category))
        #writer.save()

In [ ]:
# 1: running start on 0724 maybe at 1:45 PM (behaving normal when I get out the office)
# 2: running start on 0725 maybe at 9:45 AM
sorting_trip(df)

In [104]:
dff1 = df.head(3000000)

In [105]:
unique_bms = dff1[(dff1.SERVICE_AREA =='泰安')&(dff1.STOP == 'P')&(dff1.MVDIS_CATEGORY == 31)]['BMS_TX_BATCH'].unique()
select_vID = dff1[(dff1.BMS_TX_BATCH.isin(unique_bms))&(dff1.SERVICE_AREA == '中壢')&(dff1.STOP == 'P')]['VEHICLE_ID'].unique()

In [108]:
type(dff1[dff1.VEHICLE_ID.isin(select_vID)].groupby(['in_location', 'out_location']).size().sort_values(0, ascending=False)[:3])

pandas.core.series.Series

In [109]:
#tmp_df = pd.DataFrame(series).reset_index().drop([0], axis=1)  
series_test = dff1[dff1.VEHICLE_ID.isin(select_vID)].groupby(['in_location', 'out_location']).size().sort_values(0, ascending=False)[:3]
s_df = pd.DataFrame(series_test).reset_index()

In [111]:
s_df['in_location']

0    圓山(建國北路)
1    林口(文化北路)
2    圓山(建國北路)
Name: in_location, dtype: object

In [89]:
unique_bms_1 = df[(df.SERVICE_AREA == '泰安')&(df.STOP == 'P')&(df.MVDIS_CATEGORY == 31)]['BMS_TX_BATCH'].unique()


In [90]:
unique_bms_1[0]

1805240002310638579

In [91]:
df[(df.BMS_TX_BATCH == 1805240002310638579)&(df.STOP =='P')]

,SERVICE_AREA,BMS_TX_BATCH,DIRECTION_ID,ENTRY_TIME,EXIT_TIME,STOP,STOP_DURATION,VEHICLE_ID,IDENTITY_ID,Gender,...,O_MILEAGE,D_CITY,D_TIME,TRIP_DURATION,TRIP_DISTANCE,ENTRY_SITE,EXIT_SITE,NAME,in_location,out_location
14,泰安,1805240002310638579,S,2018-05-24 10:04:15.407,2018-05-24 10:13:22.197,P,9,12666124,E,男,...,103.4,01F2089S,2018-05-24 10:41:30.037,103,148.7,01F1572S,01F1621S,NaN,內壢,員林
518490,湖口,1805240002310638579,S,2018-05-24 09:13:02.290,2018-05-24 09:23:07.603,P,10,12666124,E,男,...,26.5,01F2089S,2018-05-24 10:41:30.037,103,153.4,01F0750S,01F0880S,NaN,內壢,員林


In [86]:
SA_name = ["Taian", "Zhongli", "Xihu", "Xilou", "Nantou", "QingShui", "Huko", "Guanxi"]
SA_Cname = ['泰安', '中壢', '西湖', '西螺', '南投', '清水', '湖口', '關西']
categories = [31, 32, 41, 42, 5]
for idx, sa in enumerate(SA_Cname) :
    ll = [y for index, y in enumerate(SA_Cname) if index != idx]
    print(ll)

['中壢', '西湖', '西螺', '南投', '清水', '湖口', '關西']
['泰安', '西湖', '西螺', '南投', '清水', '湖口', '關西']
['泰安', '中壢', '西螺', '南投', '清水', '湖口', '關西']
['泰安', '中壢', '西湖', '南投', '清水', '湖口', '關西']
['泰安', '中壢', '西湖', '西螺', '清水', '湖口', '關西']
['泰安', '中壢', '西湖', '西螺', '南投', '湖口', '關西']
['泰安', '中壢', '西湖', '西螺', '南投', '清水', '關西']
['泰安', '中壢', '西湖', '西螺', '南投', '清水', '湖口']


In [35]:
df[df.BMS_TX_BATCH.isin(df['BMS_TX_BATCH'].unique())]

,SERVICE_AREA,BMS_TX_BATCH,DIRECTION_ID,ENTRY_TIME,EXIT_TIME,STOP,STOP_DURATION,VEHICLE_ID,IDENTITY_ID,Gender,...,O_MILEAGE,D_CITY,D_TIME,TRIP_DURATION,TRIP_DISTANCE,ENTRY_SITE,EXIT_SITE,NAME,in_location,out_location
0,泰安,1805220002307810711,S,2018-05-22 09:35:57.793,2018-05-22 09:38:49.513,T,3,10287644,E,女,...,10.5,01F1664S,2018-05-22 09:41:35.427,6,17.7,01F1572S,01F1621S,NaN,三義,豐原
1,泰安,1805220002307741856,S,2018-05-22 09:35:47.530,2018-05-22 09:38:53.870,T,3,2359950,E,男,...,39.8,01F2394S,2018-05-22 10:56:45.817,110,114.7,01F1572S,01F1621S,NaN,頭份,斗南(158縣道)
2,泰安,1805220002307715213,S,2018-05-22 09:26:29.710,2018-05-22 09:29:24.643,T,3,1291031,E,男,...,71.7,01F1621S,2018-05-22 09:29:24.643,43,76.4,01F1572S,01F1621S,NaN,竹林,台中系統(連接國4)
3,泰安,1805220002307812459,S,2018-05-22 09:36:04.497,2018-05-22 09:39:00.967,T,3,913797,E,男,...,10.5,01F1725S,2018-05-22 09:45:22.653,9,23.9,01F1572S,01F1621S,NaN,三義,大雅
4,泰安,1805220002307181364,S,2018-05-22 09:36:27.147,2018-05-22 09:39:43.827,T,3,8193955,E,非自然人,...,108.4,01F1774S,2018-05-22 09:50:22.700,83,121.5,01F1572S,01F1621S,統聯汽車客運股份有限公司,機場系統(連接國2),台中(台灣大道)
5,泰安,1805220002307303887,S,2018-05-22 09:36:32.957,2018-05-22 09:39:53.047,T,3,11334686,E,非自然人,...,119.5,03F2129S,2018-05-22 10:14:04.480,129,169.0,01F1572S,01F1621S,萬泰通運有限公司,土城(連接台65),台74)
6,泰安,1805220002307314184,S,2018-05-22 09:36:40.880,2018-05-22 09:39:56.590,T,3,6058072,E,非自然人,...,64.1,01F1774S,2018-05-22 09:49:56.720,52,81.9,01F1572S,01F1621S,國光汽車客運股份有限公司,新竹(科學園區),台中(台灣大道)
7,泰安,1805220002307677300,S,2018-05-22 09:36:27.760,2018-05-22 09:39:20.840,T,3,14028676,E,男,...,93.7,01F1621S,2018-05-22 09:39:20.840,57,98.4,01F1572S,01F1621S,NaN,龍潭,台中系統(連接國4)
8,泰安,1805220002307569783,S,2018-05-22 09:36:34.440,2018-05-22 09:39:22.413,T,3,3125350,E,女,...,119.5,01F1802S,2018-05-22 09:50:15.727,102,140.1,01F1572S,01F1621S,NaN,土城(連接台65),南屯
9,泰安,1805220002307689310,S,2018-05-22 09:36:37.463,2018-05-22 09:39:36.230,T,3,7269725,E,男,...,20.5,01F1621S,2018-05-22 09:39:36.230,10,25.2,01F1572S,01F1621S,NaN,銅鑼,台中系統(連接國4)


In [69]:
df_Zhongli.groupby('STOP').size()

STOP
P     20169
T    802302
dtype: int64

In [88]:
df[(df.BMS_TX_BATCH == 1805240002310638579)&(df.STOP =='P')]

,SERVICE_AREA,BMS_TX_BATCH,DIRECTION_ID,ENTRY_TIME,EXIT_TIME,STOP,STOP_DURATION,VEHICLE_ID,IDENTITY_ID,Gender,...,O_MILEAGE,D_CITY,D_TIME,TRIP_DURATION,TRIP_DISTANCE,ENTRY_SITE,EXIT_SITE,NAME,in_location,out_location
14,泰安,1805240002310638579,S,2018-05-24 10:04:15.407,2018-05-24 10:13:22.197,P,9,12666124,E,男,...,103.4,01F2089S,2018-05-24 10:41:30.037,103,148.7,01F1572S,01F1621S,NaN,內壢,員林
518490,湖口,1805240002310638579,S,2018-05-24 09:13:02.290,2018-05-24 09:23:07.603,P,10,12666124,E,男,...,26.5,01F2089S,2018-05-24 10:41:30.037,103,153.4,01F0750S,01F0880S,NaN,內壢,員林


In [77]:
ss = df[(df.BMS_TX_BATCH == 1805240002310638579)&(df.STOP =='P')].groupby("SERVICE_AREA").size()

In [99]:
pd.DataFrame(df[(df.BMS_TX_BATCH == 1805240002310638579)&(df.STOP =='P')].groupby(["in_location", "out_location"]).size()).reset_index()['in_location']

0    內壢
Name: in_location, dtype: object

In [79]:
pd.DataFrame(ss).reset_index()

(2, 2)

In [94]:
for idx, i in pd.DataFrame(ss).reset_index().iterrows():
    print(i.SERVICE_AREA, i[0])

泰安 1
湖口 1


In [72]:
df[(df.SERVICE_AREA == '泰安')&(df.STOP == 'P')]['BMS_TX_BATCH'].unique()

array([1805240002310638579, 1805240002310299530, 1805240002310676286, ...,
       1805270002315944918, 1805260002313027664, 1805260002313634870],
      dtype=int64)

In [19]:
import sys

# These are the usual ipython objects, including this one you are creating
ipython_vars = ['In', 'Out', 'exit', 'quit', 'get_ipython', 'ipython_vars']

# Get a sorted list of the objects and their sizes
sorted([(x, sys.getsizeof(globals().get(x))) for x in dir() if not x.startswith('_') and x not in sys.modules and x not in ipython_vars], key=lambda x: x[1], reverse=True)

[('df', 5353367782),
 ('df_Zhongli', 1438690426),
 ('df_Huko', 1109981426),
 ('df_Taian', 775577090),
 ('df_Z_N', 729733304),
 ('df_Z_S', 699828410),
 ('df_QingShui', 634618047),
 ('df_Xihu', 587463328),
 ('df_Nantou', 553695737),
 ('df_Guanxi', 352333599),
 ('df_Xilou', 352333599),
 ('gnty_df', 119029),
 ('define_gantry_location_in', 136),
 ('define_gantry_location_out', 136),
 ('make_clean_gantry', 136),
 ('sorting_car_brand', 136),
 ('sorting_car_brand_inSA', 136),
 ('sorting_trip', 136),
 ('split_gantry_in', 136),
 ('split_gantry_out', 136),
 ('pd', 80)]